In [1]:
import tkinter as tk
import time

# Braille dictionary mapping each letter to its Braille representation
braille_dict = {
    'a': '100000',
    'b': '101000',
    'c': '110000',
    'd': '110100',
    'e': '100100',
    'f': '111000',
    'g': '111100',
    'h': '101100',
    'i': '011000',
    'j': '011100',
    'k': '100010',
    'l': '101010',
    'm': '110010',
    'n': '110110',
    'o': '100110',
    'p': '111010',
    'q': '111110',
    'r': '101110',
    's': '011010',
    't': '011110',
    'u': '100011',
    'v': '101011',
    'w': '011101',
    'x': '110011',
    'y': '110111',
    'z': '100111',
    ' ': '000000'
}

def convert_to_braille(text):
    braille_text = ''
    for char in text:
        braille_text += braille_dict.get(char.lower(), char) + ' '
    return braille_text

def show_braille():
    input_text = entry.get()
    braille_text = convert_to_braille(input_text)

    for braille_char in braille_text.split():
        braille_label.config(text=braille_char)
        root.update()
        time.sleep(2)

# Create the main window
root = tk.Tk()
root.title("Braille Converter")

# Entry widget for user input
entry = tk.Entry(root, width=30)
entry.pack(pady=10)

# Button to trigger conversion
convert_button = tk.Button(root, text="Convert to Braille", command=show_braille)
convert_button.pack(pady=10)

# Label to display Braille letters
braille_label = tk.Label(root, text="", font=("Courier", 24))
braille_label.pack(pady=10)

# Run the main loop
root.mainloop()

In [14]:
import tkinter as tk
from PIL import Image, ImageTk
import time

# Braille dictionary mapping each letter to its Braille representation
braille_dict = {
    'a': '100000',
    'b': '101000',
    'c': '110000',
    'd': '110100',
    'e': '100100',
    'f': '111000',
    'g': '111100',
    'h': '101100',
    'i': '011000',
    'j': '011100',
    'k': '100010',
    'l': '101010',
    'm': '110010',
    'n': '110110',
    'o': '100110',
    'p': '111010',
    'q': '111110',
    'r': '101110',
    's': '011010',
    't': '011110',
    'u': '100011',
    'v': '101011',
    'w': '011101',
    'x': '110011',
    'y': '110111',
    'z': '100111',
    ' ': '000000'
}

def convert_to_braille(text):
    braille_text = ''
    for char in text:
        braille_text += braille_dict.get(char.lower(), char) + ' '
    return braille_text

def show_braille():
    input_text = entry.get()
    braille_text = convert_to_braille(input_text)

    for braille_char in braille_text.split():
        display_braille(braille_char)
        time.sleep(2)

def display_braille(braille_pattern):
    # Create a blank image
    img = Image.new('1', (100, 160), color=255)
    pixels = img.load()

    # Fill the image based on Braille pattern
    for i in range(3):
        for j in range(2):
            index = i * 2 + j
            if index < len(braille_pattern) and braille_pattern[index] == '1':
                for x in range(40):
                    for y in range(40):
                        if (x - 20) ** 2 + (y - 20) ** 2 <= 400:
                            pixels[j * 60 + x, i * 60 + y] = 0


    # Convert the image to PhotoImage format for displaying in Tkinter
    img_tk = ImageTk.PhotoImage(img)
    braille_label.config(image=img_tk)
    braille_label.image = img_tk
    root.update()

# Create the main window
root = tk.Tk()
root.title("Braille Converter")

# Entry widget for user input
entry = tk.Entry(root, width=30)
entry.pack(pady=10)

# Button to trigger conversion
convert_button = tk.Button(root, text="Convert to Braille", command=show_braille)
convert_button.pack(pady=10)

# Label to display Braille patterns
braille_label = tk.Label(root, image=None)
braille_label.pack(pady=10)

# Run the main loop
root.mainloop()

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import pytesseract
import time

# Braille dictionary mapping each letter to its Braille representation
braille_dict = {
    'a': '100000',
    'b': '101000',
    'c': '110000',
    'd': '110100',
    'e': '100100',
    'f': '111000',
    'g': '111100',
    'h': '101100',
    'i': '011000',
    'j': '011100',
    'k': '100010',
    'l': '101010',
    'm': '110010',
    'n': '110110',
    'o': '100110',
    'p': '111010',
    'q': '111110',
    'r': '101110',
    's': '011010',
    't': '011110',
    'u': '100011',
    'v': '101011',
    'w': '011101',
    'x': '110011',
    'y': '110111',
    'z': '100111',
    ' ': '000000'
}

def convert_to_braille(text):
    braille_text = ''
    for char in text:
        braille_text += braille_dict.get(char.lower(), char) + ' '
    return braille_text

def show_braille_from_text(input_text):
    braille_text = convert_to_braille(input_text)

    for braille_char in braille_text.split():
        display_braille(braille_char)
        time.sleep(2)

def show_braille_from_camera():
    cap = cv2.VideoCapture(0)  # Use the default camera (you may need to change this)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(gray)

        if text:
            show_braille_from_text(text)

        cv2.imshow("Camera", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def display_braille(braille_pattern):
    # Create a blank image
    img = Image.new('1', (200, 200), color=255)
    pixels = img.load()

    # Fill the image based on Braille pattern
    for i in range(3):
        for j in range(2):
            index = i * 2 + j
            if index < len(braille_pattern) and braille_pattern[index] == '1':
                for x in range(40):
                    for y in range(40):
                        if (x - 20) ** 2 + (y - 20) ** 2 <= 400:
                            pixels[j * 60 + x, i * 60 + y] = 0

    # Convert the image to PhotoImage format for displaying in Tkinter
    img_tk = ImageTk.PhotoImage(img)
    braille_label.config(image=img_tk)
    braille_label.image = img_tk
    root.update()

# Create the main window
root = tk.Tk()
root.title("Braille Converter")

# Entry widget for user input
entry = tk.Entry(root, width=30)
entry.pack(pady=10)

# Button to trigger conversion from text
convert_text_button = tk.Button(root, text="Convert from Text", command=lambda: show_braille_from_text(entry.get()))
convert_text_button.pack(pady=10)

# Button to trigger conversion from camera
convert_camera_button = tk.Button(root, text="Convert from Camera", command=show_braille_from_camera)
convert_camera_button.pack(pady=10)

# Label to display Braille patterns
braille_label = tk.Label(root, image=None)
braille_label.pack(pady=10)

# Run the main loop
root.mainloop()
